<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/smallmolecules-Pradeep/bioco_smallmolecules_semiveiledet_regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [ ]:
!pip install optuna
!pip install LAMDA-SSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl

Importering av nødvendig bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev
import time

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection

# Modell
from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

In [ ]:
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content
cp: cannot stat '/content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py': No such file or directory


In [ ]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import umarkert_andel_dag
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts_semi

ModuleNotFoundError: No module named 'moduler_trening_validering_testing_evaluering'

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_semiveiledet.py /content/moduler_optimalisering_evaluering_semiveiledet.py

In [ ]:
from moduler_optimalisering_evaluering_semiveiledet import BHD
from moduler_optimalisering_evaluering_semiveiledet import SelfTrainingRandomForestRegressor

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_domene_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_enzym

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_domene_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_dag

from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_1
from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_2

from moduler_optimalisering_evaluering_semiveiledet import bhd_andel_maks
from moduler_optimalisering_evaluering_semiveiledet import andel_umarkert

# Importering av relevant data

In [ ]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/smallmolecules_data.csv',
                   header=0,
                   sep=',',
                   index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/smallmolecules_data_design.csv',
                          header=0,
                          sep=',',
                          index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [ ]:
#Tilfeldighetsfrø
random_seed = 2024

# Lagring av res

In [ ]:
# Oppretter instans for lagring av res
res_semi = Resultat_dicts_semi()

# Setter fast treningsandel og faste markerte andeler

In [ ]:
# Fast treningsandel
trenings_andel = 1.0

# Markerte andeler til undersøkelse
andel_av_markert = [0.5, 1.0]

# Parametervalg for hyperoptimalisering

In [ ]:
# Antall runder for hyperoptimalisering
n_runder_hel = 75
n_runder_andel = 75

# Antall gjentagelser
n_gjentagelser = 2

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: 20% testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _,\
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

In [ ]:
# Går igjennom andeler av satt markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andel
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  ubenyttet_andel = (1.0 - umarkert_andel)
  treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _\
  = train_test_split(trening_umarkert,
                      enzymtyper_umarkert,
                      test_size=ubenyttet_andel,
                      stratify=enzymtyper_umarkert,
                      random_state=random_seed)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  # Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
  semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt,
                          dict_res=res_semi.dict_res_1_hel,
                          rå_data=rå_data,
                          treningssett_markert=trening_markert,
                          enzymtyper_treningssett_markert=enzymtyper_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          n_splitt=n_splitt,
                          n_gjentagelser=n_gjentagelser,
                          n_runder=n_runder_andel,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)

# Lagring av resulatater
res_1_hel_semi = pd.DataFrame(res_semi.dict_res_1_hel)
res_1_hel_opt_semi = pd.DataFrame(res_semi.dict_res_1_hel_opt)

res_1_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_hel_semi.csv')
res_1_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_hel_opt_semi.csv')

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                umarkert_andel)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  # Modelloptimalisering
  semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt,
                            dict_res=res_semi.dict_res_2_hel,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            treningssett_umarkert=treningssett_umarkert,
                            treningssett=treningssett,
                            testsett=testsett,
                            n_splitt=n_splitt,
                            n_runder=n_runder_andel,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

# Lagring av resultater
res_2_hel_semi = pd.DataFrame(res_semi.dict_res_2_hel)
res_2_hel_opt_semi = pd.DataFrame(res_semi.dict_res_2_hel_opt)

res_2_hel_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_hel_semi.csv')
res_2_hel_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_hel_opt_semi.csv')

## Designproduksjon

## Alternativ 1: 80% testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Testandel
test_andel = 0.8

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _,\
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  ubenyttet_andel = (1.0 - umarkert_andel)
  treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _\
  = train_test_split(trening_umarkert,
                      enzymtyper_umarkert,
                      test_size=ubenyttet_andel,
                      stratify=enzymtyper_umarkert,
                      random_state=random_seed)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)


  # Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
  semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt,
                            dict_res=res_semi.dict_res_1_design,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            enzymtyper_treningssett_markert=enzymtyper_markert,
                            treningssett_umarkert=treningssett_umarkert,
                            enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                            treningssett=treningssett,
                            testsett=testsett,
                            n_splitt=n_splitt,
                            n_gjentagelser=n_gjentagelser,
                            n_runder=n_runder_andel,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

# Lagring av resulatater
res_1_design_semi = pd.DataFrame(res_semi.dict_res_1_design)
res_1_design_opt_semi = pd.DataFrame(res_semi.dict_res_1_design_opt)

res_1_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_design_semi.csv')
res_1_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_design_opt_semi.csv')

[I 2024-04-22 20:26:46,972] A new study created in memory with name: no-name-d4765988-5d3a-480e-bcdc-0d26f5cf70fa
[I 2024-04-22 20:39:08,606] Trial 0 finished with value: 773.5818846632824 and parameters: {'k1': 18, 'k2': 14, 'p1': 7, 'p2': 7, 'max_iters': 84, 'pool_size': 123}. Best is trial 0 with value: 773.5818846632824.
[I 2024-04-22 20:45:44,168] A new study created in memory with name: no-name-7789f484-46ad-434a-91d5-43b0ddf9a098
[I 2024-04-22 20:46:29,680] Trial 0 finished with value: 744.7789457467345 and parameters: {'n_estimators': 665, 'criterion': 'friedman_mse', 'max_depth': 28, 'min_samples_leaf': 2, 'max_features': 0.6373427447180475, 'maks_iterasjoner': 10, 'std_terskel': 0.029886293317025428}. Best is trial 0 with value: 744.7789457467345.
[I 2024-04-22 20:47:44,856] A new study created in memory with name: no-name-4b2e5912-0342-41d8-8d42-379377e7806c
[I 2024-04-22 20:48:16,205] Trial 0 finished with value: 1627.7980521794639 and parameters: {'k': 1069, 'graf_vektet_h

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# testandel
test_andel = 0.8

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

In [ ]:
# Går igjennom andeler av markerte andeler
for andel in andel_av_markert:

  # Finner tilsvarende umarkerte andeler
  umarkert_andel = andel_umarkert(andel, trening_markert, trening_umarkert)

  # Filtrer umarkert sett
  treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                umarkert_andel)

  # Treningssett
  treningssett = pd.concat([trening_markert, treningssett_umarkert],
                          axis=0)
  treningssett = treningssett.sort_index(ascending=True)


  # Antall splitt etter minste enxymtype
  n_splitt = n_splitt_enzym(rå_data, trening_markert)

  #Modelloptimalisering
  semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt,
                        dict_res=res_semi.dict_res_2_design,
                        rå_data=rå_data,
                        treningssett_markert=trening_markert,
                        treningssett_umarkert=treningssett_umarkert,
                        treningssett=treningssett,
                        testsett=testsett,
                        n_splitt=n_splitt,
                        n_runder=n_runder_andel,
                        test_andel=test_andel,
                        trenings_andel=trenings_andel,
                        random_seed=random_seed)

# Lagring av resultater
res_2_design_semi = pd.DataFrame(res_semi.dict_res_2_design)
res_2_design_opt_semi = pd.DataFrame(res_semi.dict_res_2_design_opt)

res_2_design_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_design_semi.csv')
res_2_design_opt_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_design_opt_semi.csv')

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

### Fast testandel

In [ ]:
#test_andel
test_andel = 0.2

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_hel_opt_NIR,
                         dict_res=res_semi.dict_res_1_hel_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_1_hel_semi_NIR = pd.DataFrame(res_semi.dict_res_1_hel_NIR)
res_1_hel_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_1_hel_opt_NIR)

res_1_hel_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_hel_semi_NIR.csv')
res_1_hel_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_hel_opt_semi_NIR.csv')

Beste modell

## Alternativ 2: Manuell fordeling

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_hel_opt_NIR,
                         dict_res=res_semi.dict_res_2_hel_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

res_2_hel_semi_NIR = pd.DataFrame(res_semi.dict_res_2_hel_NIR)
res_2_hel_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_2_hel_opt_NIR)

res_2_hel_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_hel_semi_NIR.csv')
res_2_hel_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_hel_opt_semi_NIR.csv')

Beste modell

## Designproduksjon

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(dict_res_opt=res_semi.dict_res_1_design_opt_NIR,
                         dict_res=res_semi.dict_res_1_design_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_gjentagelser=n_gjentagelser,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_1_design_semi_NIR = pd.DataFrame(res_semi.dict_res_1_design_NIR)
res_1_design_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_1_design_opt_NIR)

res_1_design_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_design_semi_NIR.csv')
res_1_design_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_1_design_opt_semi_NIR.csv')

Beste modell

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

# Antall splitt etter minste enxymtype
n_splitt = n_splitt_enzym(rå_data, trening_markert)

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(dict_res_opt=res_semi.dict_res_2_design_opt_NIR,
                         dict_res=res_semi.dict_res_2_design_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         n_splitt=n_splitt,
                         n_runder=n_runder_hel,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

# Lagring av resultater
res_2_design_semi_NIR = pd.DataFrame(res_semi.dict_res_2_design_NIR)
res_2_design_opt_semi_NIR = pd.DataFrame(res_semi.dict_res_2_design_opt_NIR)

res_2_design_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_design_semi_NIR.csv')
res_2_design_opt_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_res_2_design_opt_semi_NIR.csv')

Beste modell